In [40]:
import numpy as np
import pandas as pd

In [41]:
df=pd.read_csv("ml-100k/item_cc_genre.csv")

In [42]:
df['director']=df["director"].astype(str)
df['director']=df['director'].apply(lambda x: x.lower().replace(" ",""))


In [43]:
df['actor1']=df["actor1"].astype(str)
df['actor1']=df['actor1'].apply(lambda x: x.lower().replace(" ",""))
#ensuring Case variation in the actor's name is not identified as a different actor 

In [44]:
df['actor2']=df["actor2"].astype(str)
df['actor2']=df['actor2'].apply(lambda x: x.lower().replace(" ",""))

In [45]:
df['director'] = df['director'].apply(lambda x: x+" "+x+" "+x )
#adding weight to director by replicating the direcor name thrice

In [46]:
df['year']=df['year'].astype(str)

In [47]:
df['hybrid'] = df['director'] +" "+ df['actor1'] +" "+ df['actor2'] +" "+ df['genre']+" "+df['year']

In [48]:
df['hybrid']=df['hybrid'].astype(str)

In [49]:
df['hybrid'][0]


"johnlasseter johnlasseter johnlasseter tomhanks timallen Animation Children's Comedy 1995"

Paramters considered for content based filtering
The name of the director is used 3 times to add more weight to director feature

In [50]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

In [51]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=0,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [52]:
count_matrix = count.fit_transform(df['hybrid'])
count_matrix

<1681x9105 sparse matrix of type '<class 'numpy.int64'>'
	with 19634 stored elements in Compressed Sparse Row format>

In [53]:
from scipy.sparse import csc_matrix
csc = csc_matrix(count_matrix)
pd.DataFrame(csc.todense()).head()

,0,1,2,3,4,5,6,7,8,9,...,9095,9096,9097,9098,9099,9100,9101,9102,9103,9104
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
print(df.shape)

(1681, 9)


The count_matrix is a class numpy matrix with 1681 rows which is the number of unique movies and 9105 columns where 
each column is a distinct feature as identified by Count Vectorizer.



In [55]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(len(cosine_sim),len(cosine_sim[0]))

1681 1681


Each cell in the cosine_sim gives the extent of similarity between the two movies

In [72]:
titles = df['movie_title']
indices = pd.Series(df.index, index=df['movie_title'])
indices["Toy Story"]

KeyError: 'Toy Story'

In [67]:
def get_recommendations(title):
   # try:
        #title=title.lower().strip()
        idx = indices[title]
        sim_scores = list(enumerate(cosine_sim[idx])) 
        #cosine_sim[idx] is the row of cosine_sim matrix of that particular movie
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:31]
        movie_indices = [i[0] for i in sim_scores]
        d=df[df["movie_title"]==title]["director"].to_string().split()[1]
        a1=df[df["movie_title"]==title]["actor1"].to_string().split()[1]
        a2=df[df["movie_title"]==title]["actor2"].to_string().split()[1]
       
        return titles.iloc[movie_indices].head(15),d,a1,a2
    
    #except:
        #return "movie not in the database"
    

In [68]:
get_recommendations("Toy Story")

KeyError: 'Toy Story'

In [66]:
print(indices)

movie_title
Toy Story                                            0
GoldenEye                                            1
Four Rooms                                           2
Get Shorty                                           3
Copycat                                              4
Shanghai Triad (Yao a yao yao dao waipo qiao)        5
Twelve Monkeys                                       6
Babe                                                 7
Dead Man Walking                                     8
Richard III                                          9
Seven (Se7en)                                       10
Usual Suspects, The                                 11
Mighty Aphrodite                                    12
Postino, Il                                         13
Mr. Holland's Opus                                  14
French Twist (Gazon maudit)                         15
From Dusk Till Dawn                                 16
White Balloon, The                                  1